In [1]:
import string

import numpy as np
import scipy
import pandas as pd
import re

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.matutils import corpus2csc
from sklearn.feature_extraction.text import CountVectorizer

import html

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
data = pd.read_json('venv/data/presidential_speeches.json') # load data
data.sort_values(by=['date'], inplace=True) # sort by data
data.reset_index(drop=True, inplace=True) # reset index

In [3]:
# sort presidents to their parties
president_list = []
for i in range(len(data['president'].unique())):
    president_list.append(data['president'].unique()[i])
parties_list= ['Federalist', 'Democratic-Republican', 'National Republican', 'Democratic', 'Whig', 'Republican', 'Democratic (Union)']
sort_party = [0, 0, 1, 1, 1, 2, 3, 3, 4, 4, 3, 4, 4, 5, 3, 3, 6, 5, 5, 5, 5, 3, 5, 5, 5, 5, 3, 5, 5, 5, 3, 3, 5, 5, 3, 3, 5, 5, 3, 5, 3, 5, 3, 5, 3]
party = []
for i in range(len(sort_party)):
    party.append(parties_list[sort_party[i]])
data['party'] = pd.Series(dtype='string')
for i in range(len(president_list)):
    data['party'][data['president'] == president_list[i]] = party[i]
# assign each speech its respective era
data['era'] = pd.Series(dtype='string')
era_list = ['Early Republic', 'Jacksonian Democracy', 'Sectional Conflict', 'Gilded Age', 'Progressive Era', 'Depression and World Conflict', 'Social Change and Soviet Relations', 'Globalization']
for i in range(len(data)):
    if data['date'][i] < pd.Timestamp('1829-01-01T12'):
        data['era'][i] =  era_list[0]
    if pd.Timestamp('1829-01-01T12') <= data['date'][i] < pd.Timestamp('1853-01-01T12'):
        data['era'][i] = era_list[1]
    elif pd.Timestamp('1853-01-01T12') <= data['date'][i] < pd.Timestamp('1881-01-01T12'):
        data['era'][i] = era_list[2]
    elif pd.Timestamp('1881-01-01T12') <= data['date'][i] < pd.Timestamp('1897-01-01T12'):
        data['era'][i] = era_list[3]
    elif pd.Timestamp('1897-01-01T12') <= data['date'][i] < pd.Timestamp('1921-01-01T12'):
        data['era'][i] = era_list[4]
    elif pd.Timestamp('1921-01-01T12') <= data['date'][i] < pd.Timestamp('1961-01-01T12'):
        data['era'][i] = era_list[5]
    elif pd.Timestamp('1961-01-01T12') <= data['date'][i] < pd.Timestamp('1989-01-01T12'):
        data['era'][i] = era_list[6]
    elif pd.Timestamp('1989-01-01T12') <= data['date'][i]:
        data['era'][i] = era_list[7]

In [4]:
data['transcript'].replace(to_replace='(Applause.)', regex=True, value='', inplace=True)
data['transcript'].replace(to_replace='(Laughter.)', regex=True, value='', inplace=True)

In [5]:
for i in range(len(data)):
    data['transcript'][i] = data['transcript'][i].replace('\'', '')

C:\Users\Enno\AppData\Local\Temp/ipykernel_16628/1438919703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['transcript'][i] = data['transcript'][i].replace('\'', '')


In [6]:
import string
import re
import contractions

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from bs4 import BeautifulSoup
from textblob import TextBlob
from unidecode import unidecode

In [7]:
def lemmatize_pos_tagged_text(text, lemmatizer, pos_tag_dict):
  sentences = nltk.sent_tokenize(text)
  new_sentences = []

  for sentence in sentences:
    sentence = sentence.lower()
    new_sentence_words = []
    #one pos_tuple for sentence
    pos_tuples = nltk.pos_tag(nltk.word_tokenize(sentence)) 

    for word_idx, word in enumerate(nltk.word_tokenize(sentence)):
      nltk_word_pos = pos_tuples[word_idx][1]
      wordnet_word_pos = pos_tag_dict.get(
                          nltk_word_pos[0].upper(), None)
      if wordnet_word_pos is not None:
        new_word = lemmatizer.lemmatize(word, wordnet_word_pos)
      else:
        new_word = lemmatizer.lemmatize(word)

      new_sentence_words.append(new_word)

    new_sentence = " ".join(new_sentence_words)
    new_sentences.append(new_sentence)

  return " ".join(new_sentences)

In [8]:
def download_if_non_existent(res_path, res_name):
  try:
    nltk.data.find(res_path)
  except LookupError:
    print(f'resource {res_path} not found. Downloading now...')
    nltk.download(res_name)
class NltkPreprocessingSteps:
  def __init__(self, X):
    self.X = X
    download_if_non_existent('corpora/stopwords', 'stopwords')
    download_if_non_existent('tokenizers/punkt', 'punkt')
    download_if_non_existent('taggers/averaged_perceptron_tagger',
                             'averaged_perceptron_tagger')
    download_if_non_existent('corpora/wordnet', 'wordnet')
    download_if_non_existent('corpora/omw-1.4', 'omw-1.4')

    self.sw_nltk = stopwords.words('english')
    new_stopwords = ['<*>']
    self.sw_nltk.extend(new_stopwords)
    self.sw_nltk.remove('not')

    self.pos_tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

    # '!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~' 32 punctuations in python
    # we dont want to replace . first time around
    self.remove_punctuations = string.punctuation.replace('.','')

  def remove_html_tags(self):
    self.X = self.X.apply(
            lambda x: BeautifulSoup(x, 'html.parser').get_text())
    return self

  def replace_diacritics(self):
    self.X = self.X.apply(
            lambda x: unidecode(x, errors="preserve"))
    return self

  def to_lower(self):
    self.X = np.apply_along_axis(lambda x: x.lower(), self.X)
    return self

  def expand_contractions(self):
    self.X = self.X.apply(
            lambda x: " ".join([contractions.fix(expanded_word) 
                        for expanded_word in x.split()]))
    return self

  def remove_numbers(self):
    self.X = self.X.apply(lambda x: re.sub(r'\d+', '', x))
    return self

  def replace_dots_with_spaces(self):
    self.X = self.X.apply(lambda x: re.sub("[.]", " ", x))
    return self

  def remove_punctuations_except_periods(self):
    self.X = self.X.apply(
                 lambda x: re.sub('[%s]' %
                  re.escape(self.remove_punctuations), '' , x))
    return self

  def remove_all_punctuations(self):
    self.X = self.X.apply(lambda x: re.sub('[%s]' %
                          re.escape(string.punctuation), '' , x))
    return self

  def remove_double_spaces(self):
    self.X = self.X.apply(lambda x: re.sub(' +', ' ', x))
    return self

  def fix_typos(self):
    self.X = self.X.apply(lambda x: str(TextBlob(x).correct()))
    return self

  def remove_stopwords(self):
    # remove stop words from token list in each column
    self.X = self.X.apply(
            lambda x: " ".join([ word for word in x.split() 
                     if word not in self.sw_nltk]) )
    return self

  def lemmatize(self):
    lemmatizer = WordNetLemmatizer()
    self.X = self.X.apply(lambda x: lemmatize_pos_tagged_text(
                           x, lemmatizer, self.pos_tag_dict))
    return self

  def get_processed_text(self):
    return self.X

In [ ]:
data